In [1]:
#!pip3 install "psycopg[binary]"

In [2]:
import config
import psycopg
import pytz
import pandas as pd
import numpy as np
from psycopg.rows import dict_row

In [3]:
def pgDictToConn(secretDict):
    pgStrs = []
    for key in secretDict:
        pgStrs.append('{}={}'.format(key, secretDict[key]))
    return ' '.join(pgStrs)


In [4]:
#Init
pgConnStr = pgDictToConn(config.pgSecrets)

In [5]:
def getTicker(symbol):
    with psycopg.connect(pgConnStr) as conn:
        with conn.cursor(row_factory=dict_row) as cur:
            stmt = '''SELECT * FROM alpaca_minute WHERE symbol = %s'''
            data = (symbol, )
            result = cur.execute(stmt, data).fetchall()
            desc = cur.description
            cols = [col[0] for col in desc]
            conn.commit()
    df = pd.DataFrame(data=result, columns=cols)
#     df['datetime'] = df['datetime'].dt.tz_convert('America/New_York')
    df['datetime'] = df['datetime'].dt.tz_convert('America/New_York')
    return df #pd.DataFrame(data=result, columns=cols)

In [6]:
def getTickerList():
    with psycopg.connect(pgConnStr) as conn:
        with conn.cursor(row_factory=dict_row) as cur:
            stmt = '''SELECT DISTINCT symbol FROM alpaca_minute ORDER BY symbol'''
            result = cur.execute(stmt).fetchall()
            conn.commit()
            return [row['symbol'] for row in result]

In [76]:
def getFilteredTickerList(lowest_price=0.0, highest_price=999999999):
    with psycopg.connect(pgConnStr) as conn:
        with conn.cursor(row_factory=dict_row) as cur:
            stmt = '''CREATE TEMP TABLE last_tranx 
                ON COMMIT DROP
                AS
                SELECT symbol, max(datetime) AS latest FROM alpaca_minute
                GROUP BY symbol 
                '''
            cur.execute(stmt)
            stmt = '''SELECT AM.symbol FROM alpaca_minute AS AM, last_tranx AS LT
                WHERE AM.symbol = LT.symbol AND AM.datetime = LT.latest
                AND AM.close >= %s AND AM.close <= %s AND '''
            data = (lowest_price, highest_price)
            result = cur.execute(stmt, data).fetchall()
            conn.commit()
            return [row['symbol'] for row in result]

In [72]:
#Count Number of All Tickers
len(getTickerList())

1776

In [78]:
#Count Tickers Between 10 USD and 30 USD at latest close price
len(getFilteredTickerList(lowest_price=10.0, highest_price=20.0))

490

In [52]:
#Get List of All Tickers
#print(getTickerList())

['AAC', 'AACG', 'AACI', 'AACIU', 'AAC.U', 'AADI', 'AADR', 'AAIC', 'AAIC.PRB', 'AAIC.PRC', 'AAIN', 'AAL', 'AAME', 'AAM.PRA', 'AAM.PRB', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPB', 'AAPD', 'AAPL', 'AAPU', 'AAT', 'AAWW', 'AAXJ', 'AB', 'ABB', 'ABBV', 'ABC', 'ABCB', 'ABCL', 'ABCM', 'ABEO', 'ABNB', 'ABOS', 'ABR', 'ABR.PRD', 'ABR.PRE', 'ABR.PRF', 'ABSI', 'ABST', 'ABT', 'ABUS', 'ABVC', 'AC', 'ACA', 'ACAB', 'ACABU', 'ACAC', 'ACACU', 'ACAD', 'ACAH', 'ACAHU', 'ACAX', 'ACAXR', 'ACAXU', 'ACB', 'ACBA', 'ACBAU', 'ACCD', 'ACCO', 'ACDC', 'ACEL', 'ACER', 'ACET', 'ACGL', 'ACGLN', 'ACGLO', 'ACGN', 'ACHC', 'ACHL', 'ACHR', 'ACHV', 'ACI', 'ACIU', 'ACIW', 'ACLS', 'ACLX', 'ACM', 'ACMR', 'ACN', 'ACNB', 'ACNT', 'ACON', 'ACRE', 'ACR.PRD', 'ACT', 'ACTG', 'ADC.PRA', 'ADNT', 'ADX', 'ADXN', 'AEE', 'AESI', 'AEYE', 'AEZS', 'AFARU', 'AGL', 'AGZD', 'AHG', 'AHT.PRD', 'AIBBR', 'AIF', 'AIG', 'AIR', 'AJRD', 'AKTS', 'ALBT', 'ALDX', 'ALG', 'ALIT', 'ALK', 'ALLE', 'ALLG', 'ALL.PRH', 'ALL.PRI', 'ALLR', 'ALR', 'ALRN', 'ALSN', 'ALTG', 'A

In [100]:
#Sample 10 random stocks
tickers = getTickerList()
random_pick= list(np.random.choice(tickers, 10 if len(tickers) > 10 else  len(tickers), replace=False))
random_pick

['GNSS',
 'SF',
 'GSQB.U',
 'GTEC',
 'GNL.PRB',
 'GSBD',
 'AAPL',
 'GNPX',
 'GSUN',
 'GNT.PRA']

In [101]:
getTicker(random_pick[0])

,id,date,symbol,open,close,high,low,trade_count,vol,vwap,datetime
0,286908,2022-09-15,GNSS,3.1851,3.1851,3.1851,3.1851,1.0,335.0,3.185100,2022-09-15 10:39:00-04:00
1,286909,2022-09-15,GNSS,3.1350,3.1350,3.1350,3.1350,2.0,1253.0,3.135156,2022-09-15 11:12:00-04:00
2,286910,2022-09-15,GNSS,3.1050,3.1050,3.1050,3.1050,2.0,104.0,3.105000,2022-09-15 11:24:00-04:00
3,286911,2022-09-15,GNSS,3.0300,3.0300,3.0300,3.0300,1.0,100.0,3.030000,2022-09-15 12:32:00-04:00
4,286912,2022-09-15,GNSS,3.0000,3.0000,3.0000,3.0000,2.0,104.0,3.000962,2022-09-15 12:54:00-04:00
...,...,...,...,...,...,...,...,...,...,...,...
3893,290801,2023-03-13,GNSS,3.4500,3.4500,3.4500,3.4500,6.0,235.0,3.450308,2023-03-13 15:55:00-04:00
3894,290802,2023-03-13,GNSS,3.4700,3.4700,3.4700,3.4700,2.0,120.0,3.469836,2023-03-13 15:56:00-04:00
3895,290803,2023-03-13,GNSS,3.4500,3.4500,3.4500,3.4500,4.0,247.0,3.450000,2023-03-13 15:57:00-04:00
3896,290804,2023-03-13,GNSS,3.4500,3.4500,3.4500,3.4500,8.0,328.0,3.450030,2023-03-13 15:59:00-04:00


In [93]:
#Get Specific Ticker
AAPL = getTicker('AAPL')

In [71]:
AAPL.loc[390,:]

id                                   393
date                          2022-09-15
symbol                              AAPL
open                              151.89
close                             151.73
high                              151.89
low                               151.73
trade_count                        163.0
vol                              20313.0
vwap                          151.822464
datetime       2022-09-15 16:56:00-04:00
Name: 390, dtype: object